#### Прогоняем сгенерированные микшером сигналы через шумодав

In [1]:
!deepFilter ../data/mixed/**.wav -o ../data/deepfilter_out

/home/alexey/miniconda3/envs/dsp/lib/python3.10/site-packages/df/io.py:9: UserWarning: `torchaudio.backend.common.AudioMetaData` has been moved to `torchaudio.AudioMetaData`. Please update the import path.
  from torchaudio.backend.common import AudioMetaData
2023-12-17 16:36:39 | INFO     | DF | Running on torch 2.1.2+cu121
2023-12-17 16:36:39 | INFO     | DF | Running on host LAPTOP-61UF9EOA
fatal: not a git repository (or any of the parent directories): .git
2023-12-17 16:36:39 | INFO     | DF | Loading model settings of DeepFilterNet3
2023-12-17 16:36:39 | INFO     | DF | Using DeepFilterNet3 model at /home/alexey/.cache/DeepFilterNet/DeepFilterNet3
2023-12-17 16:36:39 | INFO     | DF | Initializing model `deepfilternet3`
2023-12-17 16:36:39 | INFO     | DF | Found checkpoint /home/alexey/.cache/DeepFilterNet/DeepFilterNet3/checkpoints/model_120.ckpt.best with epoch 120
2023-12-17 16:36:39 | INFO     | DF | Running on device cpu
2023-12-17 16:36:39 | INFO     | DF | Model loaded
20

#### Считаем метрики 

##### Аналитические метрики

In [2]:
from torchmetrics.functional.audio import signal_distortion_ratio, scale_invariant_signal_distortion_ratio
import torch
from pathlib import Path
import pandas as pd
import soundfile

In [3]:
SAMPLE_RATE = 48000

test_signal, test_file_rate = soundfile.read('../data/test/gt.wav')

In [4]:
path2mixed = Path('../data/deepfilter_out/')

files = []
sdrs = []
si_sdrs = []


for file in path2mixed.glob('*.wav'):
    mixed_signal, _ = soundfile.read(file)

    files.append(file.parts[-1])
    sdrs.append(signal_distortion_ratio(torch.Tensor(mixed_signal), torch.Tensor(test_signal)).item())
    si_sdrs.append(scale_invariant_signal_distortion_ratio(torch.Tensor(mixed_signal), torch.Tensor(test_signal)).item())

analytic_table = pd.DataFrame({
    'file' : files,
    'SDR' : sdrs,
    'SI-SDR' : si_sdrs
})

#### NISQA

In [5]:
!python ../NISQA/run_predict.py --mode predict_dir --pretrained_model ../NISQA/weights/nisqa.tar --data_dir ../data/deepfilter_out/  --output_dir ../nisqa_out_deepfilter/

Device: cpu
Model architecture: NISQA_DIM
Loaded pretrained model from ../NISQA/weights/nisqa.tar
# files: 4
---> Predicting ...
                          deg  mos_pred  noi_pred  dis_pred  col_pred  loud_pred   model
mixed_10dB_DeepFilterNet3.wav  4.499349  4.233055  4.534035  4.262223   4.432996 NISQAv2
mixed_-5dB_DeepFilterNet3.wav  2.827175  2.780746  3.558901  3.208273   3.495159 NISQAv2
 mixed_5dB_DeepFilterNet3.wav  4.249021  4.064727  4.410172  4.217672   4.344702 NISQAv2
 mixed_0dB_DeepFilterNet3.wav  3.583063  3.107609  4.219398  3.949718   4.022005 NISQAv2


#### DNSMOS

In [6]:
%cd ../DNS-Challenge/DNSMOS/
!python dnsmos_local.py -t ../../data/deepfilter_out/ -o ../../dnsmos_out/DNSMOS_results_deepfilter.csv
%cd ../../notebooks

/home/alexey/dsp/DNS-Challenge/DNSMOS


/home/alexey/miniconda3/envs/dsp/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


100%|███████████████████████████████████████████| 4/4 [00:00<00:00, 4029.11it/s]
4it [00:01,  3.23it/s]
/home/alexey/dsp/notebooks


#### Cоединяем все вместе

In [7]:
nisqa_table = pd.read_csv('../nisqa_out_deepfilter/NISQA_results.csv').iloc[:, :-1]
dnsmos_table = pd.read_csv('../dnsmos_out/DNSMOS_results_deepfilter.csv')[['filename', 'P808_MOS']]

dnsmos_table.filename = dnsmos_table.filename.apply(lambda x: x.split('/')[-1])

In [8]:
result_processed = pd.merge(analytic_table, nisqa_table, left_on='file', right_on='deg')
result_processed = pd.merge(result_processed, dnsmos_table, left_on='file', right_on='filename')

result_processed.drop(['filename', 'deg'], axis=1, inplace=True)
result_processed.to_csv('../metrics/metrics_processed.csv')

In [9]:
result = pd.read_csv('../metrics/metrics.csv')
result_processed.file = result_processed.file.apply(lambda x: x.replace('_DeepFilterNet3', ''))
result = pd.merge(result, result_processed, on='file', suffixes=('', '_deepfilter'))

In [10]:
result.to_csv('../metrics/metrics_all.csv')

In [11]:
result

,file,SDR,SI-SDR,mos_pred,noi_pred,dis_pred,col_pred,loud_pred,P808_MOS,SDR_deepfilter,SI-SDR_deepfilter,mos_pred_deepfilter,noi_pred_deepfilter,dis_pred_deepfilter,col_pred_deepfilter,loud_pred_deepfilter,P808_MOS_deepfilter
0,mixed_5dB.wav,2.975620,2.969908,1.465870,1.280670,4.336345,3.153045,2.823355,2.493290,11.319896,11.096474,4.249021,4.064727,4.410171,4.217672,4.344702,3.824382
1,mixed_-5dB.wav,-7.030280,-7.053289,0.627106,1.359880,2.740413,1.221595,1.424671,2.290628,2.135242,1.137534,2.827175,2.780746,3.558901,3.208273,3.495159,3.360744
2,mixed_0dB.wav,-2.028577,-2.038438,0.863872,1.335018,3.543309,2.061063,1.896784,2.432672,7.250568,6.819046,3.583063,3.107609,4.219398,3.949718,4.022005,3.402583
3,mixed_10dB.wav,7.978998,7.974598,2.225692,1.343056,4.428701,3.637326,3.289922,2.863240,14.971176,14.806463,4.499349,4.233055,4.534035,4.262223,4.432996,3.990213


#### Выводы

1) Перцептульный метрики в частоности mos пердсказанный NISQA и DNSMOS улучшились
2) Аналитические метрики в виде SDR и SI-SDR тоже показали прирост